<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a6186faf83c0fe29cf82c3ef7c205865bdd03ea1c493a4f716ddf6e4412ac89a
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1


    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-04 13:19:25
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15


-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.44 C
-------------------
Today PnL: 17.49 K (0.12%)
Current PnL: -24.33 L (-15.72%)
CY Booked + Current PnL: -10.04 L (-6.49%)
-------------------
Total profit:  1.36 L
Total loss:  -25.69 L
-------------------
Total Booked + Current PnL: 16.94 L (13.25%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (9.93%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 88.96 L (61.8%)
Deployed:  1.28 C
Current:  1.44 C
CAGR/XIRR %: 6.77%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.12,-7.65,8.31,0.02,13396.0,-13359.0,161200.0,146.09,68.0,M-SC,15.57,234.0,-1.00,1.16,40.22,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.06,-16.75,20.24,0.09,16980.0,-16885.0,83892.0,94.66,37.0,M-SC,2.75,253.0,-0.99,0.60,10.22,OX40N,NTT,DURABLES
43,ITC,452.00,0.26,-1.89,12.56,10.44,24663.0,-3780.0,196358.0,-41.56,42.0,X-LC,1.59,5.0,-0.15,1.41,3.57,X40,NTT,FMCG
35,ICICIGI,2252.93,0.44,5.69,13.72,20.19,25278.0,9922.0,184242.0,-17.20,47.0,X-MC,6.61,68.0,0.39,1.33,21.67,X40,ATH,INSURANCE
50,MASFIN,398.61,-3.23,-5.17,28.71,22.05,26674.0,-5070.0,92910.0,-18.25,49.0,H-SC,6.84,164.0,-0.19,0.67,35.05,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,838.00,4.99,-44.28,212.57,74.15,193766.0,-72451.0,91154.0,-264.07,43.0,H-SC,19.99,149.0,-0.37,0.66,16.84,XR,NTT,CHEMICALS
13,BSOFT,831.70,2.66,-21.32,91.64,50.79,101418.0,-29983.0,110670.0,0.89,75.0,H-SC,5.55,151.0,-0.30,0.80,28.86,XR,ATH,IT
15,CAMS,4762.00,2.54,3.71,20.32,24.79,53077.0,9352.0,261208.0,-1.02,56.0,X-SC,2.16,86.0,0.18,1.88,27.71,X40N,NTT,MISC
29,HAPPSTMNDS,1480.71,2.32,-36.49,191.08,84.85,155524.0,-46773.0,81392.0,-24.49,54.0,H-SC,14.02,147.0,-0.30,0.59,4.46,AR,ATH,IT
40,INDUSINDBK,1800.00,2.24,-36.08,107.89,32.89,53247.0,-27854.0,49353.0,-709.47,65.0,L-MC,6.22,259.0,-0.52,0.36,35.92,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,EASEMYTRIP,26.40,-5.00,-57.01,238.90,45.70,181600.0,-100800.0,76015.0,-20.98,51.0,M-SC,23.55,205.0,-0.56,0.55,9.56,XY24,NTT,TRAVEL
44,JCHAC,2282.00,-3.38,-33.70,50.82,-0.01,43062.0,-43071.0,84734.0,17247.83,8.0,M-SC,0.97,233.0,-1.00,0.61,0.00,OX40N,NTT,AC
50,MASFIN,398.61,-3.23,-5.17,28.71,22.05,26674.0,-5070.0,92910.0,-18.25,49.0,H-SC,6.84,164.0,-0.19,0.67,35.05,XR,ATH,FINANCE
28,GREENPANEL,537.00,-2.55,-36.40,125.07,43.14,145622.0,-66646.0,116432.0,160.98,27.0,M-SC,7.17,240.0,-0.46,0.84,9.52,XY24,NTT,MISC
69,SURYODAY,216.00,-1.90,-21.26,56.26,23.03,79324.0,-38076.0,140995.0,53.51,44.0,H-SC,10.05,167.0,-0.48,1.01,39.89,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.68,-0.58,107.93,106.72,132345.0,-715.0,122621.0,-50.82,38.0,H-SC,1.57,139.0,-0.01,0.88,22.66,AR,ATH,MISC
37,IEX,219.00,-0.86,0.18,48.36,48.63,96868.0,353.0,200307.0,-33.40,62.0,H-SC,14.74,136.0,0.00,1.44,11.75,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,0.53,-0.89,36.84,35.63,76523.0,-1856.0,207718.0,-14.97,45.0,H-MC,3.40,119.0,-0.02,1.49,15.64,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.22,-5.84,120.82,107.91,167959.0,-8628.0,139016.0,-23.39,34.0,M-SC,11.30,216.0,-0.05,1.00,0.22,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,1.12,-7.65,8.31,0.02,13396.0,-13359.0,161200.0,146.09,68.0,M-SC,15.57,234.0,-1.00,1.16,40.22,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.06,-16.75,20.24,0.09,16980.0,-16885.0,83892.0,94.66,37.0,M-SC,2.75,253.0,-0.99,0.60,10.22,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.23,-23.30,47.95,13.47,99175.0,-62838.0,206829.0,-68.51,33.0,H-SC,2.36,158.0,-0.63,1.49,5.17,XY24,NTT,PAINTS
66,SIS,528.00,0.12,-24.37,60.34,21.26,50669.0,-27060.0,83972.0,1980.83,47.0,H-SC,4.05,166.0,-0.53,0.60,13.45,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.90,5.40,63.44,72.27,113300.0,9146.0,178594.0,-9.71,69.0,M-LC,3.07,99.0,0.08,1.28,11.91,XR,NTT,IT
37,IEX,219.0,-0.86,0.18,48.36,48.63,96868.0,353.0,200307.0,-33.40,62.0,H-SC,14.74,136.0,0.00,1.44,11.75,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.90,5.40,63.44,72.27,113300.0,9146.0,178594.0,-9.71,69.0,M-LC,3.07,99.0,0.08,1.28,11.91,XR,NTT,IT
37,IEX,219.00,-0.86,0.18,48.36,48.63,96868.0,353.0,200307.0,-33.40,62.0,H-SC,14.74,136.0,0.00,1.44,11.75,XR,NTT,MISC
38,INDIAMART,4810.62,-0.68,-0.58,107.93,106.72,132345.0,-715.0,122621.0,-50.82,38.0,H-SC,1.57,139.0,-0.01,0.88,22.66,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.53,-0.89,36.84,35.63,76523.0,-1856.0,207718.0,-14.97,45.0,H-MC,3.40,119.0,-0.02,1.49,15.64,AR,ATH,PHARMA
25,FINCABLES,1641.55,0.22,-5.84,120.82,107.91,167959.0,-8628.0,139016.0,-23.39,34.0,M-SC,11.30,216.0,-0.05,1.00,0.22,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.60,-21.16,88.30,48.45,139585.0,-42426.0,158080.0,-26.75,12.0,X-MC,9.41,64.0,-0.30,1.14,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.86,-37.49,116.31,35.20,93545.0,-48243.0,80427.0,5.42,24.0,X-SC,14.39,92.0,-0.52,0.58,0.45,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.79,-11.59,37.52,21.58,46488.0,-16236.0,123903.0,-55.40,26.0,X-MC,5.74,56.0,-0.35,0.89,9.69,X40N,ATH,IT
8,AWL,485.00,-0.30,-21.89,96.52,53.50,227492.0,-66057.0,235694.0,-62.23,27.0,X-MC,1.50,58.0,-0.29,1.70,4.69,XY24,NTT,FMCG
23,DMART,5391.45,0.18,-6.93,37.65,28.12,73732.0,-14572.0,195835.0,-19.69,30.0,X-LC,4.21,19.0,-0.20,1.41,15.06,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-1.11,-23.53,114.65,64.15,208629.0,-55981.0,181970.0,-55.44,42.0,X-SC,1.48,82.0,-0.27,1.31,2.08,XY24,BTT,CEMENT
8,AWL,485.0,-0.30,-21.89,96.52,53.50,227492.0,-66057.0,235694.0,-62.23,27.0,X-MC,1.50,58.0,-0.29,1.70,4.69,XY24,NTT,FMCG
43,ITC,452.0,0.26,-1.89,12.56,10.44,24663.0,-3780.0,196358.0,-41.56,42.0,X-LC,1.59,5.0,-0.15,1.41,3.57,X40,NTT,FMCG
15,CAMS,4762.0,2.54,3.71,20.32,24.79,53077.0,9352.0,261208.0,-1.02,56.0,X-SC,2.16,86.0,0.18,1.88,27.71,X40N,NTT,MISC
36,ICICIPRULI,790.0,0.32,1.66,28.83,30.97,51798.0,2930.0,179668.0,-22.29,49.0,X-MC,2.21,75.0,0.06,1.29,14.43,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.60,-21.16,88.30,48.45,139585.0,-42426.0,158080.0,-26.75,12.0,X-MC,9.41,64.0,-0.30,1.14,0.00,X40N,ATH,FINANCE
55,QUESS,424.00,-1.20,-30.96,106.89,42.84,47974.0,-20124.0,44882.0,-54.82,35.0,X-SC,37.32,83.0,-0.42,0.32,0.00,XY24,NTT,MISC
11,BATAINDIA,2096.00,-0.86,-37.49,116.31,35.20,93545.0,-48243.0,80427.0,5.42,24.0,X-SC,14.39,92.0,-0.52,0.58,0.45,X40,NTT,FOOTWEAR
54,PGHH,17907.65,0.86,-4.78,40.47,33.76,77387.0,-9600.0,191220.0,-32.78,37.0,X-MC,4.64,57.0,-0.12,1.38,0.86,X40,ATH,FMCG
19,COLPAL,3726.84,0.94,-19.44,75.65,41.51,160507.0,-51195.0,212170.0,-4.10,37.0,X-MC,7.35,61.0,-0.32,1.53,0.94,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.20,-30.96,106.89,42.84,47974.0,-20124.0,44882.0,-54.82,35.0,X-SC,37.32,83.0,-0.42,0.32,0.00,XY24,NTT,MISC
59,RELAXO,1176.00,0.81,-46.97,187.21,52.31,144109.0,-68183.0,76977.0,-43.57,39.0,X-SC,5.32,91.0,-0.47,0.55,3.04,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.86,-37.49,116.31,35.20,93545.0,-48243.0,80427.0,5.42,24.0,X-SC,14.39,92.0,-0.52,0.58,0.45,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.79,-11.59,37.52,21.58,46488.0,-16236.0,123903.0,-55.40,26.0,X-MC,5.74,56.0,-0.35,0.89,9.69,X40N,ATH,IT
51,MEDANTA,1486.00,-1.48,-0.27,24.41,24.08,30905.0,-344.0,126606.0,-8.38,37.0,X-SC,5.94,89.0,-0.01,0.91,19.29,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,1.82,-12.02,35.58,19.28,108296.0,-41584.0,304372.0,-23.30,70.0,X-LC,6.53,1.0,-0.38,2.19,12.52,X40,ATH,IT
41,INFY,2275.00,0.96,8.50,42.74,54.87,145093.0,26591.0,339479.0,-15.65,67.0,X-LC,2.98,2.0,0.18,2.44,17.48,X40,BTT,IT
75,TMPV,600.00,-0.18,-15.59,68.44,42.18,159678.0,-43099.0,233311.0,-24.30,32.0,X-LC,3.55,3.0,-0.27,1.68,1.06,XY24,NTT,AUTO
81,VBL,671.64,1.40,-2.19,38.71,35.67,119583.0,-6922.0,308920.0,-13.89,62.0,X-LC,5.29,4.0,-0.06,2.22,11.36,X40N,ATH,FMCG
43,ITC,452.00,0.26,-1.89,12.56,10.44,24663.0,-3780.0,196358.0,-41.56,42.0,X-LC,1.59,5.0,-0.15,1.41,3.57,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,2.24,-36.08,107.89,32.89,53247.0,-27854.0,49353.0,-709.47,65.0,L-MC,6.22,259.0,-0.52,0.36,35.92,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.42,-15.44,114.83,81.67,91095.0,-14480.0,79330.0,7127.78,43.0,L-SC,16.39,271.0,-0.16,0.57,53.63,XR,NTT,CERAMICS
50,MASFIN,398.61,-3.23,-5.17,28.71,22.05,26674.0,-5070.0,92910.0,-18.25,49.0,H-SC,6.84,164.0,-0.19,0.67,35.05,XR,ATH,FINANCE
13,BSOFT,831.70,2.66,-21.32,91.64,50.79,101418.0,-29983.0,110670.0,0.89,75.0,H-SC,5.55,151.0,-0.30,0.80,28.86,XR,ATH,IT
69,SURYODAY,216.00,-1.90,-21.26,56.26,23.03,79324.0,-38076.0,140995.0,53.51,44.0,H-SC,10.05,167.0,-0.48,1.01,39.89,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.12,-7.65,8.31,0.02,13396.0,-13359.0,161200.0,146.09,68.0,M-SC,15.57,234.0,-1.00,1.16,40.22,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.18,3.42,16.95,20.95,44133.0,8606.0,260374.0,2.32,79.0,X-LC,17.94,31.0,0.20,1.87,40.27,X40,ATH,PAINTS
13,BSOFT,831.70,2.66,-21.32,91.64,50.79,101418.0,-29983.0,110670.0,0.89,75.0,H-SC,5.55,151.0,-0.30,0.80,28.86,XR,ATH,IT
40,INDUSINDBK,1800.00,2.24,-36.08,107.89,32.89,53247.0,-27854.0,49353.0,-709.47,65.0,L-MC,6.22,259.0,-0.52,0.36,35.92,XR,NTT,BANKS
32,HCLTECH,1922.01,1.24,7.78,15.73,24.73,41015.0,18822.0,260746.0,12.34,68.0,X-LC,5.21,7.0,0.46,1.88,24.20,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.42
1,25,45.32
2,50,76.23


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.17
MC    29.53
LC    24.31
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.95
X40      21.92
X40N     12.18
XR        9.92
AR        9.27
XY25      9.23
OX40N     7.71
SR        1.03
MH        0.80
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.35
X-MC    22.92
X-LC    20.83
M-SC    12.05
X-SC     8.37
H-MC     4.85
M-MC     1.40
L-SC     1.40
M-LC     1.28
H-LC     1.20
L-LC     1.00
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.10,-6.00,40.28
IT,12.54,-15.67,78.20
FINANCE,9.46,-18.05,67.94
MISC,7.59,-23.05,73.46
ELECTRICAL,6.15,-9.69,49.87
PAINTS,6.14,-8.10,24.16
INSURANCE,4.44,-2.44,38.54
PHARMA,3.99,-0.55,32.73
AUTO,3.41,-21.37,73.40


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3154666.0,21
AR,1282565.0,10
XR,1282550.0,13
X40,991136.0,14
X40N,755972.0,9
OX40N,720181.0,10
XY25,359332.0,6
SR,277528.0,2
MH,72454.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3595994.0,25
M-SC,1384035.0,15
X-MC,1298429.0,16
X-LC,867871.0,11
X-SC,756481.0,8
H-MC,403344.0,3
L-SC,266108.0,3
M-LC,113300.0,1
H-LC,66519.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1260342.0      6
           AR         883281.0      5
M-SC       XY24       792955.0      6
H-SC       XR         769447.0      7
X-MC       X40        449614.0      7
           XY24       386941.0      3
X-LC       X40        363200.0      5
H-SC       OX40N      332942.0      4
M-SC       OX40N      300068.0      5
X-SC       X40N       290651.0      3
           XY24       287508.0      3
H-SC       SR         277528.0      2
X-MC       X40N       272006.0      4
X-LC       XY24       232948.0      2
H-MC       AR         209372.0      2
           XY24       193972.0      1
X-LC       X40N       193315.0      2
X-MC       XY25       189868.0      2
L-SC       XR         178937.0      2
X-SC       X40        178322.0      2
M-SC       XR         167619.0      2
           AR         123393.0      2
M-LC       XR         113300.0      1
L-SC       OX40N       87171.0      1
X-LC       XY25        78408.0      2
H-SC       MH          72454.0      1
H-LC       AR          66519.0      1
L-MC       XR          53247.0      1
M-MC       XY25        51171.0      1
L-LC       XY25        39885.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 39.0 seconds
